In [1]:
!pip install --upgrade pip
!pip install scikit-learn
!pip install torch torchvision torchaudio
!pip install scikit-learn
!pip install tqdm
!pip install matplotlib
!pip install seaborn
!pip install pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 15.7 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 95.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 MB 148.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.2/797.2 MB 107.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 140.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 138.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 155.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 63.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 126.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 159.6 MB/s eta 0:00:0000:0100:

In [61]:
print(os.cpu_count())

80


In [62]:


import os
import torch
import numpy as np



In [7]:
import torch
import torch.nn as nn
import torchvision.models as models
! pip install timm
import timm

class HybridModel(nn.Module):
    def __init__(self, num_classes):
        super(HybridModel, self).__init__()

        # Load ResNet152 and remove the last two layers (global average pooling and FC)
        resnet = models.resnet152(pretrained=True)
        resnet.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)  # Adjusted for 1 channel

        self.resnet = nn.Sequential(*list(resnet.children())[:-2])

        # Load Xception and remove the last two layers (global average pooling and FC)
        xception = timm.create_model('xception', pretrained=True)
        xception.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=2, padding=1, bias=False)  # Adjusted for 1 channel

        self.xception = nn.Sequential(*list(xception.children())[:-2])

        # Convolutional layer to reduce the spatial dimensions
        self.conv_reduce = nn.Conv2d(4096, 2048, kernel_size=3, stride=2, padding=1)

        # Coordinate Attention module
        self.coord_att = CoordAtt(2048)

        # Flatten layer
        self.flatten = nn.Flatten()

        # Fully connected layers
        self.fc1 = nn.Linear(2048 * 16* 16, 512)  # Adjusted input size
        self.bn1 = nn.BatchNorm1d(512)
        self.fc2 = nn.Linear(512, 256)
        self.bn2 = nn.BatchNorm1d(256)
        self.fc3 = nn.Linear(256, num_classes)

    def forward(self, x):


        resnet_features = self.resnet(x)
        xception_features = self.xception(x)
        concat_features = torch.cat((resnet_features, xception_features), dim=1)

        reduced_features = self.conv_reduce(concat_features)
        coord_att_features = self.coord_att(reduced_features)
      #  print("coord size:", coord_att_features.size())

        flatten_features = self.flatten(coord_att_features)
       # print("flatten size:", flatten_features.size())
        x = self.fc1(flatten_features)
        x = self.bn1(x)
        x = nn.functional.relu(x)
        x = self.fc2(x)
        x = self.bn2(x)
        x = nn.functional.relu(x)
        x = self.fc3(x)

        return x

class CoordAtt(nn.Module):
    def __init__(self, inp, reduction=32):
        super(CoordAtt, self).__init__()
        self.pool_h = nn.AdaptiveAvgPool2d((16, 1))  # Adjusted pooling size
        self.pool_w = nn.AdaptiveAvgPool2d((1, 16))  # Adjusted pooling size
        mip = max(8, inp // reduction)
        self.conv1 = nn.Conv2d(inp, mip, kernel_size=1, stride=1, padding=0)  # Adjusted input channels
        self.bn1 = nn.BatchNorm2d(mip)
        self.act = nn.ReLU(inplace=True)
        self.conv_h = nn.Conv2d(mip, inp, kernel_size=1, stride=1, padding=0)  # Adjusted output channels
        self.conv_w = nn.Conv2d(mip, inp, kernel_size=1, stride=1, padding=0)  # Adjusted output channels

    def forward(self, x):
        identity = x
        n, c, h, w = x.size()
        x_h = self.pool_h(x)
        x_w = self.pool_w(x).permute(0, 1, 3, 2)
        y = torch.cat([x_h, x_w], dim=2)
        y = self.conv1(y)
        y = self.bn1(y)
        y = self.act(y)



        # Adjust split sizes to match the actual size after pooling and convolution
        x_h, x_w = torch.split(y, [16, 16], dim=2)  # Match split sizes to pooling size
        x_w = x_w.permute(0, 1, 3, 2)
        a_h = self.conv_h(x_h)


        a_w = self.conv_w(x_w)
        out = identity * a_h.sigmoid() * a_w.sigmoid()

        return out


model = HybridModel(num_classes=2)
model = nn.DataParallel(model)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 34.9 MB/s eta 0:00:00


/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet152_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet152_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet152-394f9c45.pth" to /home/ucloud/.cache/torch/hub/checkpoints/resnet152-394f9c45.pth
100%|██████████| 230M/230M [00:01<00:00, 184MB/s]  
/opt/conda/lib/python3.10/site-packages/timm/models/_factory.py:117: UserWarning: Mapping deprecated model name xception to current legacy_xception.
  model = create_fn(
Downloading

In [12]:
model.load_state_dict(torch.load('hybridmodel (5066249)/runs1/best.pt'))
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
model.eval()


/tmp/ipykernel_198/2962188353.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('hybridmodel (5066249)/runs1/best.pt'))


DataParallel(
  (module): HybridModel(
    (resnet): Sequential(
      (0): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (4): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu): ReLU(inplace=True)
   

In [17]:
pip install onnx


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 111.9 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [18]:
dummy_input = torch.randn(1, 1, 1024, 1024).to(device)
onnx_file_path = "ELClassifcationbased.onnx"

# Export the model
torch.onnx.export(
    model.module,                      
    dummy_input,                
    onnx_file_path,            
    export_params=True,        
    opset_version=16,          
    do_constant_folding=True,   
    input_names=['input'],     
    output_names=['output']     
)

In [20]:
import onnx

# Load the ONNX model
onnx_model = onnx.load("ELClassifcationbased.onnx")


In [21]:
def count_parameters(onnx_model):
    # Get all initializers (weights and biases) in the model
    initializers = onnx_model.graph.initializer
    
    total_params = 0
    for tensor in initializers:
        # Each tensor's size is stored in its shape attribute
        tensor_size = 1
        for dim in tensor.dims:
            tensor_size *= dim
        total_params += tensor_size
    
    return total_params

# Count parameters in the ONNX model
num_params = count_parameters(onnx_model)
print(f"Total number of parameters: {num_params}")


Total number of parameters: 423308714


In [22]:
def get_total_parameters(model):
    return sum(p.numel() for p in model.parameters())

def get_trainable_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

# Calculate parameters
total_params = get_total_parameters(model)
trainable_params = get_trainable_parameters(model)
print(f"Total number of parameters in original model: {total_params}")
print(f"Number of trainable parameters in original model: {trainable_params}")

Total number of parameters in original model: 423410282
Number of trainable parameters in original model: 423410282


In [59]:

import tensorrt as trt
print(trt.__version__)


10.3.0


In [24]:
!pip install nvidia-pyindex
!pip install tensorrt


  Preparing metadata (setup.py) ... done
  Created wheel for nvidia-pyindex: filename=nvidia_pyindex-1.0.9-py3-none-any.whl size=8419 sha256=635368ce3d0613c755514e715743bcfdef6d36bf6ee760d5e71545657a565e52
  Stored in directory: /home/ucloud/.cache/pip/wheels/2c/af/d0/7a12f82cab69f65d51107f48bcd6179e29b9a69a90546332b3
Successfully built nvidia-pyindex
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for tensorrt: filename=tensorrt-10.3.0-py2.py3-none-any.whl size=16337 sha256=524774a515357b4ccbfe25a1efdd87185d5c3060efcecb1e6baf67edcdce872a
  Stored in directory: /tmp/pip-ephem-wheel-cache-cydjyeb2/wheels/e8/a8/32/bacdf7c14f627d34b90ce8586631e2f196e3fb0fd4eb9add55
  Created wheel for tensorrt-cu12: filename=tensorrt_cu12-10.3.0-py2.py3-none-any.whl size=17554 sha256=ec16aa09598439997a76ee9767b9d50970e7ed24ae75b6bed046758785523be9
  Stored in directory: /tmp/pip-ephem

In [48]:
import onnx

model = onnx.load('ELClassifcationbased.onnx')
onnx.checker.check_model(model)


In [63]:
import tensorrt as trt

# Initialize the logger and builder
logger = trt.Logger(trt.Logger.VERBOSE)

builder = trt.Builder(logger)

# Create the network with explicit batch flag
network = builder.create_network(1 << int(trt.NetworkDefinitionCreationFlag.EXPLICIT_BATCH))

# Parse the ONNX model
parser = trt.OnnxParser(network, logger)
onnx_file_path = 'ELClassifcationbased.onnx'
with open(onnx_file_path, 'rb') as model:
    if not parser.parse(model.read()):
        print("ONNX parsing errors:")
        for i in range(parser.num_errors):
            print(f"ONNX parsing error: {parser.get_error(i).desc()}")

# Create builder config
config = builder.create_builder_config()

# Set workspace size
#config.set_memory_pool_limit(trt.MemoryPoolType.WORKSPACE, 4 << 30)  # 4 GB

# Enable FP16 mode 
if builder.platform_has_fast_fp16:
    config.set_flag(trt.BuilderFlag.FP16)
   


# Enable sparse weights
config.set_flag(trt.BuilderFlag.SPARSE_WEIGHTS)

# Build the engine
try:
    serialized_engine = builder.build_serialized_network(network, config)
    if serialized_engine is None:
        print("Failed to create the engine")
    else:
        engine_file_path = 'ELClassifcationbased_fp16.engine'
        with open(engine_file_path, 'wb') as f:
            f.write(serialized_engine)
        print(f"TensorRT engine saved as {engine_file_path}")
except Exception as e:
    print(f"Error during engine creation: {e}")



TensorRT engine saved as ELClassifcationbased_fp16.engine


In [64]:
import tensorrt as trt

# Initialize the logger and builder
logger = trt.Logger(trt.Logger.VERBOSE)

builder = trt.Builder(logger)

# Create the network with explicit batch flag
network = builder.create_network(1 << int(trt.NetworkDefinitionCreationFlag.EXPLICIT_BATCH))

# Parse the ONNX model
parser = trt.OnnxParser(network, logger)
onnx_file_path = 'ELClassifcationbased.onnx'
with open(onnx_file_path, 'rb') as model:
    if not parser.parse(model.read()):
        print("ONNX parsing errors:")
        for i in range(parser.num_errors):
            print(f"ONNX parsing error: {parser.get_error(i).desc()}")

# Create builder config
config = builder.create_builder_config()

# Set workspace size
config.set_memory_pool_limit(trt.MemoryPoolType.WORKSPACE, 4 << 30)  # 4 GB

# Enable INT8 mode 
if builder.platform_has_fast_int8:
    config.set_flag(trt.BuilderFlag.INT8)
   


# Optionally, enable sparse weights
config.set_flag(trt.BuilderFlag.SPARSE_WEIGHTS)

# Build the engine
try:
    serialized_engine = builder.build_serialized_network(network, config)
    if serialized_engine is None:
        print("Failed to create the engine")
    else:
        engine_file_path = 'ELClassifcationbased_int8.engine'
        with open(engine_file_path, 'wb') as f:
            f.write(serialized_engine)
        print(f"TensorRT engine saved as {engine_file_path}")
except Exception as e:
    print(f"Error during engine creation: {e}")



Failed to create the engine


In [65]:
def get_file_size(file_path):
    return os.path.getsize(file_path) / (1024 ** 2)  # Convert bytes to MB

engine_file_path = 'ELClassifcationbased_fp16.engine'

# Check size of TensorRT engine file
engine_size = get_file_size(engine_file_path)
print(f'Size of TensorRT engine: {engine_size:.2f} MB')

Size of TensorRT engine: 809.53 MB
